In [265]:
import pandas as pd

In [266]:
census = pd.read_csv('data/census_data/demo_char.csv', usecols = ['municipality'])

In [267]:
census['municipality'] = census['municipality'].str.rstrip(' ')

In [268]:
len(census)

245

In [269]:
recycle = pd.read_csv('data/MA_MSW_Collection_Data/musum19.csv', usecols = ['Municipality Name'])

In [270]:
len(recycle)

277

In [271]:
census_remain = list(set(census['municipality']) - set(recycle['Municipality Name']))

In [272]:
len(census_remain)

111

In [273]:
recycle_remain = list(set(recycle['Municipality Name']) - set(census['municipality']))

In [274]:
len(recycle_remain)

143

In [275]:
pairings = {}
no_rec_rem = len(recycle_remain)
no_cen_rem = len(census_remain)
for mun in recycle_remain:
    matches = [place for place in census_remain if mun in place]
    if len(matches) > 0:
        pairings[mun] = matches
        no_rec_rem = no_rec_rem - 1
        no_cen_rem = no_cen_rem - len(matches)

print(f"number of recycle muns left: {no_rec_rem}")
print(f"number of census muns left: {no_cen_rem}")
pairings  

number of recycle muns left: 122
number of census muns left: 81


{'Middleborough': ['Middleborough Center'],
 'Douglas': ['East Douglas'],
 'Mansfield': ['Mansfield Center'],
 'Amherst': ['North Amherst', 'Amherst Center', 'South Amherst'],
 'Pembroke': ['North Pembroke'],
 'Lakeville': ['North Lakeville'],
 'Acushnet': ['Acushnet Center'],
 'Harwich': ['Northwest Harwich',
  'Harwich Port',
  'Harwich Center',
  'East Harwich'],
 'Concord': ['West Concord'],
 'Seekonk': ['North Seekonk'],
 'Mashpee': ['Mashpee Neck'],
 'Westport': ['North Westport'],
 'Littleton': ['Littleton Common'],
 'Wareham': ['West Wareham', 'Wareham Center'],
 'Monson': ['Monson Center'],
 'Eastham': ['North Eastham', 'Easthampton'],
 'Marion': ['Marion Center'],
 'Millis': ['Millis-Clicquot'],
 'Raynham': ['Raynham Center'],
 'Shelburne': ['Shelburne Falls'],
 'Yarmouth': ['West Yarmouth', 'Yarmouth Port', 'South Yarmouth']}

function that:
1. take the individual group census data 
2. looks for key normalizing column
3. receives dictionary for combining data
4. rename any rows in list to key name
5. if list length > 1, take all rows in that list, multiply % columns by key normalization column, add values to form 1 row, divide by key normalization value


In [276]:
def index_correct(path_loc_str, map_dict):
    df = pd.read_csv(path_loc_str, index_col='municipality')
    key_col = list(df.filter(like='KEY').columns)[0]
    next_col = df.columns[1]
    for mun in map_dict:
        mapper = {place:mun for place in map_dict[mun]}
        df.rename(index=mapper, inplace=True)
        df.loc[mun,next_col:] = df.loc[mun,next_col:].multiply(df.loc[mun, key_col], axis="index")
        df = df.groupby(df.index).sum()
        df.loc[mun,next_col:] = df.loc[mun,next_col:].divide(df.loc[mun, key_col], axis="index")
        
    return df

In [277]:
# creating df for each group

groups = [
    'education_char',
    'economic_char',
    'housing_char',
    'demo_char'
]

for characteristic in groups:
    exec(f'{characteristic} = index_correct("data/census_data/{characteristic}.csv", pairings)')

In [278]:
comb_char = pd.concat([education_char,economic_char,housing_char,demo_char], axis=1)
comb_char.head()

,KEY_population_25_and_older,edu_high_school_and_higher_%,edu_bachelors_and_higher_%,KEY_pop_16_+,unemployment_rate_%,median_household_income,mean_household_income,families_below_poverty_level_%,people_below_poverty_level_%,KEY_total_housing_units,...,KEY_total_population,males-to-100females,median_age,demo_white_%,demo_black_aa_%,demo_american_indian_%,demo_asian_%,demo_native_islander%,demo_other_Race_%,demo_hispanic_latino_%
municipality,,,,,,,,,,,,,,,,,,,,,
Abington,11793,94.7,39.1,13727,4.3,99381.0,105390.0,2.5,4.1,6781,...,16436,97.7,42.2,93.2,3.0,0.3,4.9,0.0,1.4,2.6
Acushnet,2265,90.4,16.3,2651,6.7,81375.0,85990.0,6.9,8.1,1309,...,3177,93.4,45.2,97.3,3.3,3.1,0.9,0.0,1.9,9.2
Adams,4130,89.3,22.9,4560,5.3,48252.0,62772.0,13.2,14.4,3086,...,5335,109.5,43.5,97.7,2.3,0.0,0.9,0.0,0.8,1.6
Agawam,21598,93.3,33.2,23942,3.5,68944.0,87473.0,4.3,7.9,12253,...,28696,95.3,46.4,95.6,2.8,0.7,2.8,0.0,0.4,5.4
Amesbury,12633,94.2,41.1,14452,3.6,82468.0,100890.0,3.6,6.3,7549,...,17434,95.1,43.9,98.1,1.5,0.6,2.0,0.0,0.1,2.5


In [279]:
pairings2 = {}
no_rec_rem = len(recycle_remain)
no_cen_rem = len(census_remain)
for mun in census_remain:
    matches = [place for place in recycle_remain if mun in place]
    if len(matches) > 0:
        pairings2[mun] = matches
        no_rec_rem = no_rec_rem - len(matches)
        no_cen_rem = no_cen_rem - 1

print(f"number of recycle muns left: {no_rec_rem}")
print(f"number of census muns left: {no_cen_rem}")
pairings2 

number of recycle muns left: 140
number of census muns left: 109


{'Bridgewater': ['West Bridgewater', 'East Bridgewater'], 'Ware': ['Wareham']}

In [280]:
row = comb_char.loc['Bridgewater']
row.rename(index='West Bridgewater', inplace=True)
comb_char.append(row)

,KEY_population_25_and_older,edu_high_school_and_higher_%,edu_bachelors_and_higher_%,KEY_pop_16_+,unemployment_rate_%,median_household_income,mean_household_income,families_below_poverty_level_%,people_below_poverty_level_%,KEY_total_housing_units,...,KEY_total_population,males-to-100females,median_age,demo_white_%,demo_black_aa_%,demo_american_indian_%,demo_asian_%,demo_native_islander%,demo_other_Race_%,demo_hispanic_latino_%
municipality,,,,,,,,,,,,,,,,,,,,,
Abington,11793.0,94.700000,39.100000,13727.0,4.30000,99381.000000,105390.000000,2.500000,4.100000,6781.0,...,16436.0,97.7000,42.200000,93.200000,3.000000,0.300000,4.900000,0.00000,1.400000,2.600000
Acushnet,2265.0,90.400000,16.300000,2651.0,6.70000,81375.000000,85990.000000,6.900000,8.100000,1309.0,...,3177.0,93.4000,45.200000,97.300000,3.300000,3.100000,0.900000,0.00000,1.900000,9.200000
Adams,4130.0,89.300000,22.900000,4560.0,5.30000,48252.000000,62772.000000,13.200000,14.400000,3086.0,...,5335.0,109.5000,43.500000,97.700000,2.300000,0.000000,0.900000,0.00000,0.800000,1.600000
Agawam,21598.0,93.300000,33.200000,23942.0,3.50000,68944.000000,87473.000000,4.300000,7.900000,12253.0,...,28696.0,95.3000,46.400000,95.600000,2.800000,0.700000,2.800000,0.00000,0.400000,5.400000
Amesbury,12633.0,94.200000,41.100000,14452.0,3.60000,82468.000000,100890.000000,3.600000,6.300000,7549.0,...,17434.0,95.1000,43.900000,98.100000,1.500000,0.600000,2.000000,0.00000,0.100000,2.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woburn,29255.0,94.300000,45.400000,33526.0,3.00000,91022.000000,108751.000000,4.400000,6.300000,16872.0,...,40304.0,102.3000,40.000000,81.600000,7.400000,0.400000,9.800000,0.40000,3.500000,4.600000
Woods Hole,724.0,98.900000,81.200000,741.0,2.40000,97569.000000,188991.000000,7.700000,9.800000,1053.0,...,816.0,42.2000,61.800000,96.700000,0.000000,0.000000,1.000000,0.00000,3.300000,0.000000
Worcester,121514.0,84.700000,30.200000,154224.0,5.60000,48139.000000,68613.000000,14.600000,20.000000,78438.0,...,185143.0,96.3000,34.700000,72.600000,15.600000,1.200000,8.300000,0.20000,6.500000,21.900000


In [281]:
comb_char.loc['Bridgewater'].rename(index='East Bridgewater', inplace=True)

KEY_population_25_and_older        16935.0
edu_high_school_and_higher_%          94.3
edu_bachelors_and_higher_%            35.8
KEY_pop_16_+                       23131.0
unemployment_rate_%                    6.1
median_household_income            95675.0
mean_household_income             109501.0
families_below_poverty_level_%         6.1
people_below_poverty_level_%           9.4
KEY_total_housing_units             8531.0
occupancy_%                           95.3
1-unit_detached_%                     64.7
1-unit_attached_%                      5.8
2-units_%                              6.3
3-4_units_%                            5.2
5-9_units_%                            3.2
10-19_units_%                          7.1
20+_units_%                            5.8
mobile_home_%                          2.0
boat_RV_van_%                          0.0
owner-occupied_%                      71.6
renter-occupied_%                     28.4
KEY_total_population               27436.0
males-to-10

In [282]:
census_remain = list(set(comb_char.index) - set(recycle['Municipality Name']))
census_remain.sort()
len(census_remain)

81

In [283]:
# census_remain

In [284]:
recycle_remain = list(set(recycle['Municipality Name'])-set(comb_char.index))
recycle_remain.sort()
len(recycle_remain)

122

In [285]:
# recycle_remain

-------
census towns un-matched = 81 (don't care about too much)

recycle towns un-matched = 122 (care)

In [288]:
# check what towns in the recycle data could pull in multiple towns from the census data

pairings3 = {}
sub_towns = 0
for mun in recycle['Municipality Name']:
    matches = [place for place in comb_char.index if mun in place]
    # check if any of the matches are actually towns that are already listed in the recycle towns
    # this will also remove the 'mun' from the match list
    for match in matches.copy():
        if match in list(recycle['Municipality Name']):
            matches.remove(match)
    
    if len(matches) > 0:
        pairings3[mun] = matches
        sub_towns = sub_towns + len(matches)

print(f"number of census entries to convert: {sub_towns}")
pairings3  

number of census entries to convert: 16


{'Brookfield': ['West Brookfield'],
 'Chatham': ['West Chatham'],
 'Deerfield': ['South Deerfield'],
 'Dennis': ['Dennis Port', 'East Dennis', 'South Dennis', 'West Dennis'],
 'Duxbury': ['South Duxbury'],
 'Falmouth': ['East Falmouth', 'North Falmouth', 'West Falmouth'],
 'Marshfield': ['Marshfield Hills'],
 'Pepperell': ['East Pepperell'],
 'Plymouth': ['North Plymouth'],
 'Sandwich': ['East Sandwich'],
 'Scituate': ['North Scituate']}

In [289]:
def index_correct2(df, map_dict):
    key_col = list(df.filter(like='KEY').columns)[0]
    next_col = df.columns[1]
    for mun in map_dict:
        mapper = {place:mun for place in map_dict[mun]}
        df.rename(index=mapper, inplace=True)
        df.loc[mun,next_col:] = df.loc[mun,next_col:].multiply(df.loc[mun, key_col], axis="index")
        df = df.groupby(df.index).sum()
        df.loc[mun,next_col:] = df.loc[mun,next_col:].divide(df.loc[mun, key_col], axis="index")
        
    return df

In [290]:
# shouldn't be touching any towns that are already paired up in the recycle data
# just to check I'm going to look at the keys in the map_dict
muncheck = pairings3.keys()
# now I'll run the function
comb_char = index_correct2(comb_char,pairings3)

In [291]:
recycle_remain = list(set(recycle['Municipality Name'])-set(comb_char.index))
recycle_remain.sort()
len(recycle_remain)

122

In [292]:
census_remain = list(set(comb_char.index) - set(recycle['Municipality Name']))
census_remain.sort()
len(census_remain)

65

In [293]:
# checking what towns in the census data may be duplicated to approximated un-matched recycle towns

pairings4 = {}
sub_towns = 0
for mun in comb_char.index:
    matches = [place for place in recycle['Municipality Name'] if mun in place]
    # check if the match is already accounted for in the census data
    for match in matches.copy():
        if match in list(comb_char.index):
            matches.remove(match)
    
    if len(matches) > 0:
        pairings4[mun] = matches
        sub_towns = sub_towns + len(matches)

print(f"number of recycle municipalities to account for: {sub_towns}")
pairings4

number of recycle municipalities to account for: 8


{'Andover': ['North Andover'],
 'Attleboro': ['North Attleborough'],
 'Bridgewater': ['East Bridgewater', 'West Bridgewater'],
 'Chester': ['Chesterfield'],
 'Longmeadow': ['East Longmeadow'],
 'Marlborough': ['New Marlborough'],
 'Salem': ['New Salem']}

In [310]:
def index_correct_reverse(census_df, map_dict):
    df = census_df
    for mun in map_dict:
        for place in map_dict[mun]:
            row = comb_char.loc[mun]
            row.rename(index=place, inplace=True)
            df = df.append(row)
       
    return df

In [311]:
comb_char = index_correct_reverse(comb_char, pairings4)

added row for North Andover
added row for North Attleborough
added row for East Bridgewater
added row for West Bridgewater
added row for Chesterfield
added row for East Longmeadow
added row for New Marlborough
added row for New Salem


In [74]:
pairingscheck = {}
sub_towns = 0
for mun in comb_char.index:
    matches = [place for place in recycle['Municipality Name'] if mun in place]
    if len(matches) > 1:
        pairingscheck[mun] = matches
        sub_towns = sub_towns + len(matches)

print(f"number of recycle municipalities to account for: {sub_towns}")
pairingscheck

number of recycle municipalities to account for: 0


{}

In [75]:
pairingscheck = {}
sub_towns = 0
for mun in recycle['Municipality Name']:
    matches = [place for place in comb_char.index if mun in place]
    if len(matches) > 1:
        pairingscheck[mun] = matches
        sub_towns = sub_towns + len(matches)

print(f"number of recycle municipalities to account for: {sub_towns}")
pairingscheck

number of recycle municipalities to account for: 0


{}

In [313]:
census_remain = list(set(comb_char.index) - set(recycle['Municipality Name']))
census_remain.sort()
len(census_remain)

65

In [316]:
census_remain

['Baldwinville',
 'Bellingham',
 'Bliss Corner',
 'Bridgewater',
 'Buzzards Bay',
 'Chelsea',
 'Cochituate',
 'Cordaville',
 'Essex',
 'Everett',
 'Fiskdale',
 'Forestdale',
 'Foxborough',
 'Granby',
 'Great Barrington',
 'Green Harbor-Cedar Crest',
 'Hanscom AFB',
 'Hatfield',
 'Holbrook',
 'Holland',
 'Hopedale',
 'Hopkinton',
 'Housatonic',
 'Hull',
 'Lee',
 'Lenox',
 'Leominster',
 'Madaket',
 'Mattapoisett Center',
 'Millers Falls',
 'Monomoscoy Island',
 'Monument Beach',
 'Nahant',
 'New Seabury',
 'Norton Center',
 'Ocean Bluff-Brant Rock',
 'Ocean Grove',
 'Onset',
 'Oxford',
 'Palmer',
 'Pinehurst',
 'Pocasset',
 'Popponesset',
 'Popponesset Island',
 'Rowley',
 'Rutland',
 'Sagamore',
 'Salisbury',
 'Seabrook',
 'Seconsett Island',
 'Siasconset',
 'Smith Mills',
 'South Ashburnham',
 'South Lancaster',
 'Teaticket',
 'The Pinehills',
 'Turners Falls',
 'Vineyard Haven',
 'Ware',
 'Webster',
 'Weweantic',
 'White Island Shores',
 'Whitinsville',
 'Winthrop',
 'Woods Hole']

In [314]:
recycle_remain = list(set(recycle['Municipality Name'])-set(comb_char.index))
recycle_remain.sort()
len(recycle_remain)

114

In [315]:
recycle_remain

['Acton',
 'Aquinnah',
 'Ashby',
 'Ashfield',
 'Ashland',
 'Auburn',
 'Avon',
 'Becket',
 'Bedford',
 'Berkley',
 'Berlin',
 'Bernardston',
 'Billerica',
 'Bolton',
 'Boxborough',
 'Buckland',
 'Canton',
 'Carlisle',
 'Charlemont',
 'Chelmsford',
 'Chilmark',
 'Cohasset',
 'Colrain',
 'Conway',
 'Cummington',
 'Dalton',
 'Dartmouth',
 'Dighton',
 'Dracut',
 'Dudley',
 'Dunstable',
 'Egremont',
 'Erving',
 'Fairhaven',
 'Florida',
 'Freetown',
 'Gill',
 'Goshen',
 'Grafton',
 'Granville',
 'Halifax',
 'Hamilton',
 'Hampden',
 'Hancock',
 'Hanover',
 'Harvard',
 'Heath',
 'Hinsdale',
 'Holden',
 'Holliston',
 'Lanesborough',
 'Leicester',
 'Leverett',
 'Lincoln',
 'Ludlow',
 'Manchester',
 'Medway',
 'Merrimac',
 'Middlefield',
 'Middleton',
 'Millbury',
 'Monroe',
 'Montague',
 'Monterey',
 'Mount Washington',
 'Natick',
 'New Ashford',
 'Norfolk',
 'Norwell',
 'Oak Bluffs',
 'Otis',
 'Paxton',
 'Peru',
 'Phillipston',
 'Plainfield',
 'Plainville',
 'Rochester',
 'Rockland',
 'Rowe',
 '

In [317]:
# pip install uszipcode

In [325]:
from uszipcode import Zipcode, SearchEngine

In [326]:
search = SearchEngine()
search.by_city_and_state("natick", "ma")

Start downloading data for simple zipcode database, total size 9MB ...
  1 MB finished ...
  2 MB finished ...
  3 MB finished ...
  4 MB finished ...
  5 MB finished ...
  6 MB finished ...
  7 MB finished ...
  8 MB finished ...
  9 MB finished ...
  10 MB finished ...
  Complete!


[SimpleZipcode(zipcode='01760', zipcode_type='Standard', major_city='Natick', post_office_city='Natick, MA', common_city_list=['Natick'], county='Middlesex County', state='MA', lat=42.28, lng=-71.35, timezone='Eastern', radius_in_miles=4.0, area_code_list=['508', '774'], population=32786, population_density=2214.0, land_area_in_sqmi=14.81, water_area_in_sqmi=1.0, housing_units=14039, occupied_housing_units=13327, median_home_value=425600, median_household_income=95328, bounds_west=-71.402518, bounds_east=-71.302922, bounds_north=42.329135, bounds_south=42.247729)]

In [357]:
search.by_city_and_state("Mount Washington", "ma")

[SimpleZipcode(zipcode='01450', zipcode_type='Standard', major_city='Groton', post_office_city='Groton, MA', common_city_list=['Groton'], county='Middlesex County', state='MA', lat=42.62, lng=-71.58, timezone='Eastern', radius_in_miles=6.0, area_code_list=['978'], population=10646, population_density=325.0, land_area_in_sqmi=32.76, water_area_in_sqmi=0.99, housing_units=3989, occupied_housing_units=3753, median_home_value=435000, median_household_income=117127, bounds_west=-71.660464, bounds_east=-71.493541, bounds_north=42.663495, bounds_south=42.569386)]

In [361]:
zip_lookup = {}
inprecision = []
for town in recycle_remain:
    res = search.by_city_and_state(town, "ma")
    
    if len(res) > 1:
        print(f"{town} returned more than one zip")
    if len(res) > 0:
        
        zip_lookup[town] = res[0].values()[0]
        if res[0].values()[2] != town:
            print(f'fuzzy match for {town} was non-exact at: {res[0].values()[2]}')
            inprecision.append(town)


fuzzy match for Freetown was non-exact at: East Freetown
fuzzy match for Lanesborough was non-exact at: Lanesboro
fuzzy match for Mount Washington was non-exact at: Groton
New Ashford returned more than one zip
fuzzy match for New Ashford was non-exact at: New Bedford
fuzzy match for Peru was non-exact at: Newton Upper Falls
fuzzy match for Phillipston was non-exact at: Holliston
fuzzy match for Tyngsborough was non-exact at: Tyngsboro
fuzzy match for Washington was non-exact at: Abington
fuzzy match for Westhampton was non-exact at: Easthampton


In [364]:
inprecision.remove('Freetown')
inprecision.remove('Lanesborough')
inprecision.remove('Tyngsborough')
inprecision

['Mount Washington',
 'New Ashford',
 'Peru',
 'Phillipston',
 'Washington',
 'Westhampton']

In [365]:
for town in inprecision:
    del zip_lookup[town]

In [366]:
len(zip_lookup)

96

In [367]:
zip_lookup

{'Acton': '01720',
 'Ashby': '01431',
 'Ashfield': '01330',
 'Ashland': '01721',
 'Auburn': '01501',
 'Avon': '02322',
 'Becket': '01223',
 'Bedford': '01730',
 'Berkley': '02779',
 'Berlin': '01503',
 'Bernardston': '01337',
 'Billerica': '01821',
 'Bolton': '01740',
 'Boxborough': '01719',
 'Buckland': '01338',
 'Canton': '02021',
 'Carlisle': '01741',
 'Charlemont': '01339',
 'Chelmsford': '01824',
 'Chilmark': '02535',
 'Cohasset': '02025',
 'Colrain': '01340',
 'Conway': '01341',
 'Cummington': '01026',
 'Dalton': '01226',
 'Dighton': '02715',
 'Dracut': '01826',
 'Dudley': '01571',
 'Dunstable': '01827',
 'Erving': '01344',
 'Fairhaven': '02719',
 'Freetown': '02717',
 'Gill': '01354',
 'Goshen': '01032',
 'Grafton': '01519',
 'Granville': '01034',
 'Halifax': '02338',
 'Hampden': '01036',
 'Hanover': '02339',
 'Harvard': '01451',
 'Heath': '01346',
 'Hinsdale': '01235',
 'Holden': '01520',
 'Holliston': '01746',
 'Lanesborough': '01237',
 'Leicester': '01524',
 'Leverett': '0105

In [213]:
# For saving files

# comb_char.to_csv('data/census_data/census_sum.csv', index=True)